In [1]:
import numpy as np
import pandas as pd

# Scalable data analytics: dask.
import dask
import dask.bag as db
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

# For GC large pandas dataframes after use.
import gc

In [2]:
city = 'boston'
date_boston = ['20181011','20181117','20181213','20190117','20190209','20190312',
               '20190415','20190519','20190614','20190714','20190819','20190922',
               '20191011']
df_boston = pd.DataFrame()
for i,date in enumerate(date_boston[:-1]):
    filepath = city+'/calendar'+date+'.csv'
    df = pd.read_csv(filepath)
    df.date = df.date.apply(lambda x: int(x[0:4]+x[5:7]+x[8:]))
    df = df[df.date < int(date_boston[i+1])][['listing_id','date','available','price']]
    df_boston = pd.concat([df_boston, df], ignore_index=True)


In [3]:
city = 'cambridge'
date_cambridge = ['20181118','20181215','20190122','20190213','20190318',
                  '20190418','20190523','20190624','20190720','20190828','20190925',
                  '20191118'] # missing 201810 data
df_cambridge = pd.DataFrame()
for i,date in enumerate(date_cambridge[:-1]):
    filepath = city+'/calendar'+date+'.csv'
    df = pd.read_csv(filepath)
    df.date = df.date.apply(lambda x: int(x[0:4]+x[5:7]+x[8:]))
    df = df[df.date < int(date_cambridge[i+1])][['listing_id','date','available','price']]
    df_cambridge = pd.concat([df_cambridge, df], ignore_index=True)

In [4]:
assert (df_cambridge.date.min() == int(date_cambridge[0]))
assert (df_cambridge.date.max() == int(date_cambridge[-1])-1)
assert (df_boston.date.min() == int(date_boston[0]))
assert (df_boston.date.max() == int(date_boston[-1])-1)

assert sum(df_cambridge.duplicated(subset=['listing_id','date'])) == 0
assert sum(df_boston.duplicated(subset=['listing_id','date'])) == 0

In [5]:
# df_cambridge['city'] = 'cambridge'
# df_boston['city'] = 'boston'
df_compiled = pd.concat([df_cambridge, df_boston], ignore_index=True)

In [6]:
# only 'price' contains NaN; drop these rows
print(np.mean(df_compiled.isna()))
df_compiled = df_compiled.dropna()
assert np.sum(np.sum(df_compiled.isna())) == 0

listing_id    0.000000
date          0.000000
available     0.000000
price         0.146957
dtype: float64


In [7]:
# when handling duplicated <lisitng_id, date>, 
# prioritize unavailable and take mean price
df_compiled['unavailable'] = df_compiled.available.apply(lambda x: 0 if x=='t' else 1)
df_compiled.price = df_compiled.price.apply(lambda x: float(x[1:].replace(',','')))
df_compiled = df_compiled.groupby(['listing_id','date']).agg({'price':'mean','unavailable':'max'}).reset_index()

In [8]:
# write to csv
df_compiled.to_csv('calendar_compiled.csv')